In [1]:
import zmq
from nilearn.input_data import NiftiMasker
import pandas as pd
import numpy as np
import glob
import os
from sklearn.cluster import KMeans
import nibabel as nib

def compute_outliers_FD(target_file, source, threshold):
    mvt_file = pd.read_csv(target_file, header=None, sep='  ') 
    if source=='FSL':
        mvt_file = mvt_file.rename(columns={0: 'Rx', 1: 'Ry', 2: 'Rz', 3: 'Tx', 4: 'Ty', 5: 'Tz'})
    else:
        mvt_file = mvt_file.rename(columns={3: 'Rx', 4: 'Ry', 5: 'Rz', 0: 'Tx', 1: 'Ty', 2: 'Tz'})
    mvt_file[['Rx', 'Ry', 'Rz']] = mvt_file[['Rx', 'Ry', 'Rz']]*180/np.pi
    diff_mvt = mvt_file.diff()
    diff_mvt.iloc[0]=[0,0,0,0,0,0]
    diff_sphere = diff_mvt
    diff_sphere[['Rx', 'Ry', 'Rz']] = diff_sphere[['Rx', 'Ry', 'Rz']]*50*np.pi/180
    return diff_sphere.abs().sum(axis=1) > threshold

def reset_socket(socket):
    socket.close()            
    socket=context.socket(zmq.REQ)
    socket.connect("tcp://localhost:5555")

import struct
def send_array(socket, array):
    socket.send(array.tobytes())
    message=socket.recv()
    val=struct.unpack('b', message)[0]
    if val != 5:
        raise NameError('Unexpected response from server')

def reduce_array(socket, array):
    for f in array:
        send_array(socket, np.insert(f,0,0))

            
def get_representatives(socket):
    reps = []
    socket.send(np.asarray([11.0]))
    message = socket.recv()
    payload = np.frombuffer(message, dtype=np.double())
    if payload[0] == 6.0:
        socket.send(np.asarray([5.0]))
        for i in range(0, int(payload[1])):
            message=socket.recv()
            payload = np.frombuffer(message, dtype = np.double())
            reps.append(payload)
            socket.send(np.asarray([5.0]))
        message = socket.recv()
    return np.vstack(reps)


def get_array_representatives(array, socket):
    reduce_array(socket, array)
    reps = get_representatives(socket)
    
def prepare_confounds(mvt_path, csf_path, wm_path, gm_path, compute_mvt_derivatives=True, compute_mvt_squares=True):
    df_orig = pd.read_csv(mvt_path, header=None, sep='  ')
    df_data = df_orig.to_numpy()
    total_data = df_data.copy()
    if csf_path is not None:
        csf_signal = pd.read_csv(csf_path, header=None)
        total_data = np.hstack((csf_signal.to_numpy(), total_data))
    if wm_path is not None:
        wm_signal = pd.read_csv(wm_path, header=None)
        total_data = np.hstack((wm_signal.to_numpy(), total_data))
    if gm_path is not None:
        gm_signal = pd.read_csv(gm_path, header=None)
        total_data = np.hstack((gm_signal.to_numpy(), total_data))
    if compute_mvt_derivatives:
        derivatives = np.gradient(df_data)[0]
        total_data = np.hstack((total_data, derivatives))
    if compute_mvt_squares:
        total_data = np.hstack((total_data, df_data**2))
        if compute_mvt_derivatives:
            total_data = np.hstack((total_data, derivatives**2))
    all_confounds = pd.DataFrame(total_data)
    return all_confounds
    
# We would like for each assignement to obtain in what 'state' we were.
# To gather statistics, basically.
import seaborn as sns

def gather_stats_and_plot(hrf_delay, events, k, assignements, x):
    """
    Given an input array of events, along with an input array of CAPs assignement and their associated time array x, the number of CAPs and an HRF delay:
    - Assign to each event the assignement that is closest in time (ie: assign the time t that is closest to event time + hrf_delay)
    - Aggregate, per CAP, the different events from perspective of behavioural response (error type basically)
    - Lastly, plot the results, displaying frequency of each error type per CAP.
    :param hrf_delay: 
    :param events: 
    :param k: 
    :param assignements: 
    :param x: 
    :return: 
    """
    gathered_stats = np.zeros((4, k))
    get_array_entry = lambda i : (int(events.iloc[i]['trial_type']=='go')<<1) + int(events.iloc[i]['resp_correct'])
    for i in range(0, events.shape[0]-1):
        time = int(round(events.iloc[i]['onset'] + hrf_delay))
        corresp = np.where(x == time)
        if corresp[0].size == 0:
            print('Time {} not found'.format(time))
        centroid = assignements[np.where(x == time)]
        gathered_stats[get_array_entry(i), centroid] += 1
    cap_arr = []
    gathered_stats /= gathered_stats.sum(axis=0)
    for i in range(0,k):
        cap_arr.extend(['CAP_' + str(i)]*4)
    df=pd.DataFrame({'Frequency': gathered_stats.flatten(order='F'), 'Error Type': ['Commission Error', 'Correct Omission', 'Omission Error', 'Correct Commission']*k, 'CAP': cap_arr})
    sns.catplot(x='Error Type', hue='CAP', y='Frequency', data=df, kind='bar')
    plt.show()



def make_path(subject_name, session_name):
    return os.path.join('/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/', 'analysis', subject_name, session_name, 'func', subject_name + '_' + session_name + '_task-gradCPT_bold_masked-fullbrain_zscored_smoothed-5mm_detrended_csf-removed_wm-removed_8volumes-scrubbed.npy')



def run_k_means_and_get_assignements(k, reps, sessions):
    # Get clusters for a specific k
    # Then we can run kmeans on this reduced version
    kmeans = KMeans(n_clusters=k) # some k. A relevant question (but to be answered later) is whether consensus clustering might be relevant here or not
    kmeans.fit(reps) # compute centroids on reduced data version
    # Important: save the CAPs that were found, so they can be inspected.
    masker = NiftiMasker(mask_img='/usr/local/fsl/data/standard/MNI152_T1_2mm_brain_mask_dil1.nii.gz')
    m = nib.load(masker.mask_img)
    for i in range(0,k):
        data = m.get_fdata()
        data[m.get_fdata().astype(bool)] = reps[i]
        nib.save(nib.Nifti1Image(data, m.affine, m.header), '/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/analysis/sub-07/CAPS_' + str(i) + '.nii.gz')
    assignements = []
    # Now we want to get the assignments for all sessions, using these centroids. To do so, we must reload the data (a tad painful)
    for sess in sessions:
        session_name = sess.split('/')[-2]
        save_path = make_path(subject_name, session_name)
        # To get the session, load it from disk
        session_data = np.load(save_path)
        # Get back motion outliers
        mvt_file = glob.glob(os.path.join(sess, 'func','sub*gradCPT*_mcf.txt'))
        mvt_file = mvt_file[0]
        outliers = compute_outliers_FD(mvt_file, 'FSL', 0.5)[8:]
        # Predict only on volumes with decent motion
        assignements.append(kmeans.predict(session_data[:,:][np.logical_not(outliers),:].astype(np.double())))
        del session_data # We still care about the RAM !
    return assignements, kmeans

# Get statistics for that k and plot it

def get_event_type_id(event):
    return 2*int(event['trial_type']=='go') + int(event['resp_correct'])
    
    


def gather_stats_for_session(sess_assignements, k, sess_events, hrf_delay=5.0):
    times = np.asarray(range(8, 788))
    transition_stats = np.zeros((4, k, k))
    for e in range(0, sess_events.shape[0]):
        t = sess_events.iloc[e]['onset'] + hrf_delay
        t_low = int(t)
        t_up = t_low + 1
        id_up = np.where(times==t_up)[0]
        id_low = np.where(times==t_low)[0]
        if id_up.size == 0 or id_low.size == 0:
            print('No corresp for time interval {}-{}, one of the bounds is missing in EPI time'.format(t_low, t_up))
        else:
            # Get event type and increment count of transition
            transition_stats[get_event_type_id(sess_events.iloc[e]),sess_assignements[id_low], sess_assignements[id_up]] += 1
    return transition_stats

/home/guibertf/nilearn/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
def convert_vector_to_df(binary_vector):
    d = {}
    for i in range(0, binary_vector.size):
        if(binary_vector[i]):
            reg = np.zeros((binary_vector.size,))
            n = 'outlier_' + str(i)
            reg[i] = 1
            d[n] = reg
    return d

In [3]:
from nilearn.glm.first_level import FirstLevelModel
import matplotlib.pyplot as plt

fmri_glm = FirstLevelModel(t_r=1.0,
                           noise_model='ar1',
                           standardize=True,
                           hrf_model='glover + derivative + dispersion',
                           drift_model=None,
                           high_pass=.01, smoothing_fwhm=5, minimize_memory=True)

/home/guibertf/nilearn/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [ ]:
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
import gc

# We will get the Yeo 7 atlas, to identify signal level in somatomotor region
yeo = datasets.fetch_atlas_yeo_2011()
atlas_masker = NiftiLabelsMasker(labels_img=yeo['thick_7'], standardize=False,
                           memory='nilearn_cache')



subject_motor_scores={}
subjects='/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/preprocessed/'
for subject_path  in sorted(glob.glob(subjects + 'sub*/')):
    subject_name = subject_path.split('/')[-2]
    sessions = sorted(glob.glob(os.path.join(subject_path, 'ses*/')), key=lambda x: int(x.split('-')[3][:-1]))
    print('Starting with ' + subject_name + '...')
    behavioural_path = os.path.join('/'.join(subject_path.split('/')[:-4]), subject_name)
    behav_sessions = sorted(glob.glob(os.path.join(behavioural_path, 'ses*/')), key=lambda x: int(x.split('-')[3][:-1]))
    for i, sess in enumerate(sessions):
        session_name = sess.split('/')[-2]
        print('    Analyzing ' + session_name + '...')
        # Load the data, preprocess it, so on and so forth
        mvt_file = glob.glob(os.path.join(sess, 'func','sub*gradCPT*_mcf.txt'))
        if len(mvt_file) > 0:
            mvt_file = mvt_file[0]
            outliers = compute_outliers_FD(mvt_file, 'FSL', 0.5)
            # Prepare the confounds to be regressed out: movement parameters, csf signal, white matter signal
            confounds = prepare_confounds(mvt_path=mvt_file, csf_path= glob.glob(os.path.join(sess, 'func','sub*gradCPT*_csf.txt'))[0], wm_path= glob.glob(os.path.join(sess, 'func','sub*gradCPT*_wm.txt'))[0], gm_path=None)
            # Also add outliers to censor volumes with too much motion
            confounds = confounds.join(pd.DataFrame(convert_vector_to_df(outliers)))
            # Session data will be zscored, smoothed (5 fwhm mm), linearly detrended. 
            # To further spare RAM, the input data is restricted to a brain mask, which encompasses entire brain. To minimize odds of throwing away relevant information, we use a dilated version
            #masker = NiftiMasker(mask_img='/usr/local/fsl/data/standard/MNI152_T1_2mm_brain_mask_dil1.nii.gz', standardize='zscore', smoothing_fwhm=5, t_r=1, detrend=True)
            # Apply the processing pipeline to our input data, at last c:
            #session_data = masker.fit_transform(glob.glob(os.path.join(sess, 'func','sub*gradCPT_bold_mni.nii.gz'))[0],confounds=confounds)[8:,:].astype(np.double())
            behav_sess = behav_sessions[i]
            events=pd.read_csv(glob.glob(os.path.join(behav_sess, 'func', '*gradCPT_events.tsv'))[0], delimiter='\t')
            # fit the GLM, retaining only events which were correct
            fit_glm = fmri_glm.fit(glob.glob(os.path.join(sess, 'func','sub*gradCPT_bold_mni.nii.gz'))[0],confounds=confounds,events=events[(events['resp_correct']==1)])
            # Now prepare the contrast vector of go - no go
            design_mat = fit_glm.design_matrices_[0]
            contrast = np.zeros((design_mat.shape[1]))
            contrast[design_mat.columns.get_loc('go')]=1
            contrast[design_mat.columns.get_loc('nogo')]=-1
            # Get the contrast as a Z-score
            img=fit_glm.compute_contrast(contrast, output_type='z_score')
            # Get somatomotor value of the map
            masked_img=atlas_masker.fit_transform(img)
            somatomotor_val=masked_img[0,1]
            # Save the map to disk
            nib.save(img, os.path.join('/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/analysis', subject_name, session_name, 'func/go_minus_nogo.nii.gz'))
            print('        Saved image to ' +  os.path.join('/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/analysis', subject_name, session_name, 'func/go_minus_nogo.nii.gz'))
            # Save somatomotor value
            subject_motor_scores[subject_path + '_' + sess] = somatomotor_val
            del masked_img, fit_glm, design_mat, img
            gc.collect()

Starting with sub-01...
    Analyzing ses-01...


<ipython-input-1-e138e997fbaf>:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  mvt_file = pd.read_csv(target_file, header=None, sep='  ')
<ipython-input-1-e138e997fbaf>:62: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_orig = pd.read_csv(mvt_path, header=None, sep='  ')
/home/guibertf/nilearn/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `click` in events data will be ignored.
  warnings.warn(("Unexpected column `{}` in events "
/home/guibertf/nilearn/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `resp_correct` in events da

        Saved image to /home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/analysis/sub-01/ses-01/func/go_minus_nogo.nii.gz
    Analyzing ses-02...


<ipython-input-1-e138e997fbaf>:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  mvt_file = pd.read_csv(target_file, header=None, sep='  ')
<ipython-input-1-e138e997fbaf>:62: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_orig = pd.read_csv(mvt_path, header=None, sep='  ')
/home/guibertf/nilearn/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `click` in events data will be ignored.
  warnings.warn(("Unexpected column `{}` in events "
/home/guibertf/nilearn/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `resp_correct` in events da

In [ ]:
# We will worry about subject 07 for now, because it has no motion outliers.
subject_path = '/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/preprocessed/sub-07/'
subject_name = subject_path.split('/')[-2]
sessions = sorted(glob.glob(os.path.join(subject_path, 'ses*/')), key=lambda x: int(x.split('-')[3][:-1]))


# Now that various sessions have been 'reduced' we lack the centroids. We will compute them in two steps:
# Get the various sessions
sess = sessions[0]
session_name = sess.split('/')[-2]
# Load the data, preprocess it, so on and so forth
mvt_file = glob.glob(os.path.join(sess, 'func','sub*gradCPT*_mcf.txt'))
if len(mvt_file) > 0:
    mvt_file = mvt_file[0]
    outliers = compute_outliers_FD(mvt_file, 'FSL', 0.5)[8:]
    # Prepare the confounds to be regressed out: movement parameters, csf signal, white matter signal
    confounds = prepare_confounds(mvt_path=mvt_file, csf_path= glob.glob(os.path.join(sess, 'func','sub*gradCPT*_csf.txt'))[0], wm_path= glob.glob(os.path.join(sess, 'func','sub*gradCPT*_wm.txt'))[0], gm_path=None)
    # Session data will be zscored, smoothed (5 fwhm mm), linearly detrended. 
    # To further spare RAM, the input data is restricted to a brain mask, which encompasses entire brain. To minimize odds of throwing away relevant information, we use a dilated version
    masker = NiftiMasker(mask_img='/usr/local/fsl/data/standard/MNI152_T1_2mm_brain_mask_dil1.nii.gz', standardize='zscore', smoothing_fwhm=5, t_r=1, detrend=True)
    # Apply the processing pipeline to our input data, at last c:
    #session_data = masker.fit_transform(glob.glob(os.path.join(sess, 'func','sub*gradCPT_bold_mni.nii.gz'))[0],confounds=confounds)[8:,:].astype(np.double())


behavioural_path = os.path.join('/'.join(subject_path.split('/')[:-4]), subject_name)
behav_sessions = sorted(glob.glob(os.path.join(behavioural_path, 'ses*/')), key=lambda x: int(x.split('-')[3][:-1]))
#
import matplotlib.pyplot as plt

behav_sess = behav_sessions[0]
events=pd.read_csv(glob.glob(os.path.join(behav_sess, 'func', '*gradCPT_events.tsv'))[0], delimiter='\t')

In [ ]:
from nilearn.glm.first_level.hemodynamic_models import _hrf_kernel

# We must have an array: [custom_hrf, custom_derivatives, custom_dispersion], basically.
# One function per 

def custom_hrf(hrf_kernel_name, tr, oversampling):
    base_kernels=_hrf_kernel(hrf_kernel_name, tr, oversampling, None)[0]
    triangular_shape=np.asarray([i if i <= 0.8/0.8 else (1.6-i)/0.8 for i in np.arange(0,1.6,tr/oversampling)])
    return np.convolve(triangular_shape, base_kernels, mode='same')

In [ ]:
custom_spm = lambda tr, oversamp: custom_hrf('spm', tr, oversamp)

In [ ]:
identity = np.eye(8, 788)

In [ ]:
confounds

In [ ]:
import pandas as pd
df = pd.DataFrame({'26' : identity[0,:], '27' : identity[1,:], '28' : identity[2,:], '29' : identity[3,:], '30' : identity[4,:], '31' : identity[5,:], '32' : identity[6,:], '33' : identity[7,:]})

In [ ]:
masked_img=atlas_masker.fit_transform(img)1confounds.join(df)

In [ ]:
fit_glm = fmri_glm.fit(glob.glob(os.path.join(sess, 'func','sub*gradCPT_bold_mni.nii.gz'))[0],confounds=confounds.join(df),events=events[(events['resp_correct']==1)])

In [ ]:
from nilearn.plotting import plot_stat_map
from nilearn.image import mean_img
avg_img = mean_img(glob.glob(os.path.join(sess, 'func','sub*gradCPT_bold_mni.nii.gz'))[0])


In [ ]:
fit_glm.design_matrices_[0]

In [ ]:
design_mat = fit_glm.design_matrices_[0]
contrast = np.zeros((design_mat.shape[1]))
contrast[design_mat.columns.get_loc('go')]=1
contrast[design_mat.columns.get_loc('nogo')]=-1
img=fit_glm.compute_contrast(contrast, output_type='z_score')

In [ ]:
fit_glm.design_matrices_[0].columns.get_loc('nogo')

In [ ]:
plot_stat_map(img, bg_img=avg_img, threshold=1.1, display_mode='z', black_bg=True, title='Go - nogo contrast')

In [ ]:
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
yeo = datasets.fetch_atlas_yeo_2011()

atlas_masker = NiftiLabelsMasker(labels_img=yeo['thick_7'], standardize=False,
                           memory='nilearn_cache')

In [ ]:
masked_img=atlas_masker.fit_transform(img)

In [ ]:
masked_img

In [ ]:
yeo['colors_7']

In [ ]:
import nibabel as nib
nib.save(img,'/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/analysis/sub-07/ses-01/func/go-minus-nogo_contrast.nii.gz')

In [ ]:
from nilearn.reporting import get_clusters_table
from nilearn import input_data

table = get_clusters_table(img, stat_threshold=3.1,
                           cluster_threshold=20).set_index('Cluster ID', drop=True)
table.head()

In [ ]:
coords = table.loc[range(1, 7), ['X', 'Y', 'Z']].values

In [ ]:
masker = input_data.NiftiSpheresMasker(coords, standardize=True, detrend=True, t_r=1.0, smoothing_fwhm=5.0)

real_timeseries = masker.fit_transform(glob.glob(os.path.join(sess, 'func','sub*gradCPT_bold_mni.nii.gz'))[0], confounds=confounds)

In [ ]:
masker = input_data.NiftiSpheresMasker(coords)
predicted_timeseries = masker.fit_transform(fit_glm.predicted[0])

In [ ]:
plt.plot(predicted_timeseries[:,5], label='Predicted')
plt.plot(real_timeseries[:,5], label='Real')
plt.legend()

In [ ]:
from nilearn.glm.first_level.hemodynamic_models import _sample_condition
from nilearn.glm.first_level.experimental_paradigm import check_events
events_shifted_corr = events_shifted[events_shifted['resp_correct']==1]
trial_type, onset, duration, modulation = check_events(events_shifted_corr)
condition=events['trial_type']=='go'
exp_condition = (onset[condition], duration[condition],modulation[condition])

In [ ]:
a=np.vstack(((filtered_events['onset']-8).to_numpy(), np.zeros((filtered_events['onset'].size)),np.ones((filtered_events['onset'].size))))
np.savetxt("/home/guibertf/Documents/Freya/2021-Esterman_InOutZone/niftifiles_GVA_2020/derivatives/analysis/sub-07/ses-01/func/nogo-events.csv", a.T, delimiter=" ")

In [ ]:
fit_glm = fmri_glm.fit(glob.glob(os.path.join(sess, 'func','sub*gradCPT_bold_mni.nii.gz'))[0],confounds=confounds,events=events[(events['trial_type']=='no-go') & (events['resp_correct']==1)])

In [ ]:
plot_stat_map(img, bg_img=avg_img, threshold=1.0, display_mode='z', black_bg=True, title='No go')

In [ ]:
from nilearn.glm.first_level.hemodynamic_models import _sample_condition
from nilearn.glm.first_level.experimental_paradigm import check_events

events_corr = events[events['resp_correct']==1].iloc[1:2]
trial_type, onset, duration, modulation = check_events(events_corr)
condition=events_corr['trial_type']=='go'
exp_condition = (onset[condition], duration[condition],modulation[condition])



In [ ]:
_hrf_kernel(custom_spm, 1.0, 50, None)

In [ ]:
events[(events['trial_type']=='no-go') & (events['resp_correct']==1)]

In [ ]:
fit_glm.design_matrices_[0]

In [ ]:
plt.plot(fit_glm.design_matrices_[0]['nogo'])

In [ ]:
fit_glm.design_matrices_[0].shape

In [ ]:
contrast = np.zeros((fit_glm.design_matrices_[0].shape[1]))
contrast[0]=1
img=fit_glm.compute_contrast(contrast, output_type='z_score')

In [ ]:
go_count=0
nogo_count = 0
for i in range(0, events_shifted.shape[0]):
    t = events_shifted.iloc[i]['trial_type']
    if t== 'go':
        events_shifted.trial_type.iloc[i]=t+'_'+str(go_count)
        go_count+=1
    else:
        events_shifted.trial_type.iloc[i]=t+'_'+str(nogo_count)
        nogo_count+=1


In [ ]:
events_shifted

In [ ]:
from nilearn.glm.first_level import make_first_level_design_matrix
design_matrix = make_first_level_design_matrix(np.arange(0,788,0.8), events=events_shifted[['onset','duration','trial_type']][events_shifted['resp_correct']==1], hrf_model='spm', drift_model=None)

In [ ]:
%matplotlib notebook
from nilearn.plotting import plot_design_matrix

plot_design_matrix(design_matrix)

In [ ]:
# If we are using the custom HRF (which models triangular shapes instead of boxcar shapes), then we must shift back the onset timings by 0.8 seconds (from their max amplitude to their actual starting point)
events_shifted=events.copy()
events_shifted['onset']-=0.8
events_shifted['duration']+=1.6

In [ ]:
events

In [ ]:
fit_glm

In [ ]:
events_shifted[['onset','duration','trial_type']][events_shifted['resp_correct']==1]['trial_type']

In [ ]:
nogo_count+go_count

In [ ]:
fit_glm.design_matrices_[0].columns

In [ ]:
relevant_cols=fit_glm.design_matrices_[0].columns[:-42]
go_entries = np.zeros(fit_glm.design_matrices_[0].shape[1])
nogo_entries = np.zeros(fit_glm.design_matrices_[0].shape[1])

go_c_count = 0
nogo_c_count = 0
for i,f in enumerate(relevant_cols):
    if f[:2]=='go':
        go_entries[i] = 1
        go_c_count += 1
    else:
        nogo_entries[i]=1
        nogo_c_count += 1

In [ ]:
go_entries_single = np.zeros(fit_glm.design_matrices_[0].shape[1])
go_entries_single[0] = 1.0

In [ ]:
nogo_entries_singles = np.zeros(fit_glm.design_matrices_[0].shape[1])
nogo_entries_singles[np.where(nogo_entries)[0][0]] = 1

In [ ]:
(go_entries/go_entries.sum() - nogo_entries/nogo_entries.sum()).sum()

In [ ]:
go_col=[0]*fit_glm.design_matrices_[0].shape[1]
go_col[0]=1

nogo_col=[0]*fit_glm.design_matrices_[0].shape[1]
nogo_col[1]=1


In [ ]:
(go_entries*nogo_entries.sum() - nogo_entries*go_entries.sum()).sum()

In [ ]:
%matplotlib inline
from nilearn.plotting import plot_contrast_matrix
plot_contrast_matrix(go_entries_single - nogo_entries_singles, design_matrix=fit_glm.design_matrices_[0])
plt.show()

In [ ]:
z_map = fit_glm.compute_contrast(go_entries_single - nogo_entries_singles, output_type='z_score')